In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# List of URLs to load documents from
urls = [
    "https://lscosmetics.ma/",
    "https://baraboucle.com/blogs/journal/quel-produit-pour-mon-enfant",
    "https://www.sandrinejeannerose.fr/blog/index/billet/2311_cheveux-enfants-crepus-frises-soins-conseils",
    "https://www.ducray.com/fr-be/etats-pelliculaires/symptomes/pellicules-enfant",
    "https://www.kryneo.com/blogs/conseils-capillaires/les-7-types-d-ingredients-a-eviter-dans-les-produits-capillaires",
]

# Create a Text Splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

# List to store vector stores and retrievers for each document
vector_stores_and_retrievers = []

# Iterate over each URL, load the document, split it, and create a vector store and retriever
for url in urls:
    loader = WebBaseLoader(url)
    document = loader.load()
    splits = text_splitter.split_documents(document)
    
    # Create a Chroma vector store with OpenAIEmbeddings for each document
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()
    
    # Store information about the document, vector store, and retriever
    vector_stores_and_retrievers.append({
        'url': url,
        'original_document': document,
        'splits': splits,
        'vectorstore': vectorstore,
        'retriever': retriever
    })

# Now, vector_stores_and_retrievers contains information about each document, its splits,
# and the corresponding vector store and retriever
# You can access and use these components as needed


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Your account is not active, please check your billing details on our website..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Your account is not active, please check your billing details on our website..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Your account is not active, please check your billing details on our website..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 8.0 seconds as it raised RateLimitError: Your account is not active, please check your billing details on our website..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 10.0 seconds as it raised RateLimitError: Your account is not active, please check your billing deta

RateLimitError: Your account is not active, please check your billing details on our website.

In [19]:
#from langchain import hub
#rag_prompt = hub.pull("rlm/rag-prompt")

from langchain.prompts import PromptTemplate

template = """You are an assistant for question-answering tasks.Usually ask the user kindlly about his contact.
Try to give him the right answer as concise as possible and keep going in conversation.
If you don't know the answer, just say that you don't know, don't try to make up an answer.  
Always say "thanks for asking!" at the end of the answer. 
"""
rag_prompt_custom = PromptTemplate.from_template(template)



# Use the customized prompt
#response = rag_prompt.run(prompt=custom_prompt)

# Print or process the model's response
#print(response)


In [20]:
# LLM
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

llm = ChatOpenAI(model_name="gpt-4", temperature=0)
conversation = ConversationChain(llm=llm,memory=ConversationBufferMemory())

In [21]:
# RAG chain

from langchain.schema.runnable import RunnablePassthrough

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | rag_prompt | llm

NameError: name 'retriever' is not defined

In [12]:
rag_chain.invoke("Ces produits semblent chers.")

" Les produits proposés par LS Cosmetics sont de qualité et 100% purs et naturels. Certains produits peuvent sembler chers, mais ils sont offerts à des prix variés pour s'adapter à tous les budgets."